# Build RAI pipeline

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `Pipeline` using component defined by yaml

**Motivations** - This notebook demos how to build complex RAI sample pipeline.  

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
#import required libraries
import time

from azure.ml import MLClient, dsl
from azure.ml.entities import JobInput, load_component

## 1.2. Configure credential

We are using `DefaultAzureCredential` to get access to workspace. When an access token is needed, it requests one using multiple identities(`EnvironmentCredential, ManagedIdentityCredential, SharedTokenCacheCredential, VisualStudioCodeCredential, AzureCliCredential, AzurePowerShellCredential`) in turn, stopping when one provides a token.
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for more information.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 
Reference [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python) for all available credentials if it does not work for you.  

In [ ]:
from azure.identity import DefaultAzureCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token('https://management.azure.com/.default')
except Exception as ex:
    # If exception happens when retrieve token, try exclude the failed credential like this then try again:
    # Exclude VSCode credential:
    # credential = DefaultAzureCredential(exclude_visual_studio_code_credential=True)
    raise Exception("Failed to retrieve a token from the included credentials due to the following exception, try to add `exclude_xxx_credential=True` to `DefaultAzureCredential` and try again.") from ex

## 1.3. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. 

In [ ]:
try:
    ml_client = MLClient.from_config(credential=credential)
except Exception as ex:
    # NOTE: Update following workspace information if not correctly configure before
    client_config = {
        "subscription_id": "<SUBSCRIPTION_ID>",
        "resource_group": "<RESOURCE_GROUP>",
        "workspace_name": "<WORKSPACE_NAME>"
    }

    if client_config["subscription_id"].startswith('<'):
        print("please update your <SUBSCRIPTION_ID> <RESOURCE_GROUP> <WORKSPACE_NAME> in notebook cell")
        raise ex
    else:  # write and reload from config file
        import json, os
        config_path = "../../.azureml/config.json"
        os.makedirs(os.path.dirname(config_path), exist_ok=True)
        with open(config_path, "w") as fo:
            fo.write(json.dumps(client_config))
        ml_client = MLClient.from_config(credential=credential, path=config_path)
print(ml_client)

## 1.4. Retrieve or create an Azure Machine Learning compute target

In [ ]:
# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
try:
    ml_client.compute.get(name=cluster_name)
except Exception:
    print('Creating a new compute target...')
    from azure.ml.entities import AmlCompute
    compute = AmlCompute(
        name=cluster_name,
        size="Standard_D2_v2",
        max_instances=2
    )
    ml_client.compute.begin_create_or_update(compute)

## 1.5. Retrieve or create RAI environment

In [ ]:
rai_cli_example_path = "../../../../cli/jobs/pipelines-with-components/rai_pipeline_adult_analyse"

environment_name = "AML-RAI-Environment"
try:
    ml_client.environments.get(name=environment_name,version='1')
except Exception:
    print(f'Cannot find environment {environment_name} in workspace, creating it')
    from azure.ml.entities import Environment
    environment = Environment(
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220303.v1",
        conda_file=f"{rai_cli_example_path}/environment/conda_envs/python-aml-rai.yaml",
        name=environment_name,
        version='1',
        description="AML RAI environment"
    )
    ml_client.environments.create_or_update(environment)

## 1.6 Retrieve or create RAI input data

In [ ]:
input_train_data = "adult_train_pq"
input_test_data = "adult_test_pq"

try:
    ml_client.data.get(name=input_train_data,version='1')
except Exception:
    print(f'Cannot find data {input_train_data} in workspace, register it now')
    from azure.ml.entities import Data
    from azure.ml._constants import AssetTypes

    train_data = Data(
        path=f"{rai_cli_example_path}/data/adult_train.parquet",
        type=AssetTypes.URI_FILE,
        description="RAI sample train data",
        name= input_train_data,
        version='1'
    )

    ml_client.data.create_or_update(train_data)

try:
    ml_client.data.get(name=input_test_data,version='1')
except Exception:
    print(f'Cannot find data {input_test_data} in workspace, register it now')
    from azure.ml.entities import Data
    from azure.ml._constants import AssetTypes

    test_data = Data(
        path=f"{rai_cli_example_path}/data/adult_test.parquet",
        type=AssetTypes.URI_FILE,
        description="RAI sample test data",
        name= input_test_data,
        version='1'
    )

    ml_client.data.create_or_update(test_data)


# 2. Define command component via YAML
Below is example to define command component using YAML.

In [ ]:
rai_components_path = f"{rai_cli_example_path}/components"

train_log_reg_component = load_component(yaml_file=f"{rai_components_path}/component_train_logreg.yaml")
register_model_component = load_component(yaml_file=f"{rai_components_path}/component_register_model.yaml")
rai_constructor_component = load_component(yaml_file=f"{rai_components_path}/component_rai_insights.yaml")
rai_explanation_component = load_component(yaml_file=f"{rai_components_path}/component_explain.yaml")
rai_causal_component = load_component(yaml_file=f"{rai_components_path}/component_causal.yaml")
rai_counterfactual_component = load_component(yaml_file=f"{rai_components_path}/component_counterfactual.yaml")
rai_erroranalysis_component = load_component(yaml_file=f"{rai_components_path}/component_error_analysis.yaml")
rai_gather_component = load_component(yaml_file=f"{rai_components_path}/component_gather_rai_insights.yaml")

# 3. Sample pipeline job

## 3.1 Build pipeline

In [ ]:
@dsl.pipeline(
    compute="cpu-cluster",
    description="Submission of classification pipeline from Python",
    experiment_name=f"test_classification_pipeline_from_python",
)
def rai_classification_pipeline(
    target_column_name,
    train_data,
    test_data,
):
    train_job = train_log_reg_component(
        target_column_name=target_column_name, 
        training_data=train_data
    )

    register_job = register_model_component(
        model_input_path=train_job.outputs.model_output,
        model_base_name="test_classification_pipeline_from_python",
        model_name_suffix=-1,  # Should be default
    )

    create_rai_job = rai_constructor_component(
        title="Run built from Python",
        task_type="classification",
        model_info_path=register_job.outputs.model_info_output_path,
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
        categorical_column_names='["Race", "Sex", "Workclass", "Marital Status", "Country", "Occupation"]',
        maximum_rows_for_test_dataset=5000,  # Should be default
        classes="[]",  # Should be default
    )

    explain_job = rai_explanation_component(
        comment="Insert text here",
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )

    causal_job = rai_causal_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        treatment_features='["Age", "Sex"]',
        heterogeneity_features='["Marital Status"]',
        nuisance_model="linear",  # Should be default
        heterogeneity_model="linear",  # Should be default
        alpha=0.05,  # Should be default
        upper_bound_on_cat_expansion=50,  # Should be default
        treatment_cost="0",  # Should be default
        min_tree_leaf_samples=2,  # Should be default
        max_tree_depth=2,  # Should be default
        skip_cat_limit_checks=False,  # Should be default
        categories="auto",  # Should be default
        n_jobs=1,  # Should be default
        verbose=1,  # Should be default
        random_state="None",  # Should be default
    )

    counterfactual_job = rai_counterfactual_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        total_cfs=10,  # Bug filed - should be total_CFs
        desired_class="opposite",
        method="random",  # Should be default
        desired_range="[]",  # Should be default
        permitted_range="{}",  # Should be default
        features_to_vary="all",  # Should be default
        feature_importance=True,  # Should be default
    )

    erroranalysis_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        filter_features='["Race", "Sex"]',
        max_depth=3,  # Should be default
        num_leaves=31,  # Should be default
        min_child_samples=20,  # Should be default
    )

    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_1=explain_job.outputs.explanation,
        insight_2=causal_job.outputs.causal,
        insight_3=counterfactual_job.outputs.counterfactual,
        insight_4=erroranalysis_job.outputs.error_analysis,
    )

    rai_gather_job.outputs.dashboard.mode = "upload"
    rai_gather_job.outputs.ux_json.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
        "ux_json": rai_gather_job.outputs.ux_json,
    }

    
pipeline_job = rai_classification_pipeline(
    target_column_name="income",
    train_data=JobInput(path=f"{input_train_data}:1", mode="download"),
    test_data=JobInput(path=f"{input_test_data}:1", mode="download"),
)

# 3.2 Submit pipeline job

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(pipeline_job, experiment_name="pipeline_samples")
print(f'Job link: {pipeline_job.services["Studio"].endpoint}')
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)